![CC-BY-SA](https://mirrors.creativecommons.org/presskit/buttons/88x31/svg/by-sa.svg)


This notebook was created by [Bernardo Freitas Paulo da Costa](http://www.im.ufrj.br/bernardofpc),
and is licensed under Creative Commons BY-SA.

Antes de enviar este Teste, verifique que tudo está funcionando como esperado.
Por exemplo, **rode o código inteiro, do zero**.
Para isso, vá no menu, escolha _Kernel_, depois _Restart & Run All_.

Verifique, também, que você respondeu todas as questões:
* as questões de código têm `YOUR CODE HERE` (e você pode apagar o `raise NotImplemented` ao incluir sua resposta)
* as questões discursivas têm "YOUR ANSWER HERE".

---

**Ideia e polinômios originais**: Luan Lima

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Teste 3: Analisando o método de Newton

## Questão 1: Implementação e testes simples

Implemente abaixo a função `newton_pts`, que executa o método de Newton,
retornando todos os pontos percorridos pelo algoritmo até atingir algum critério de parada.
Utilize como critérios de parada `xtol`, `ytol` e `maxiter`.

In [ ]:
def newton_pts(f, df, x, xtol=1e-8, ytol=1e-8, maxiter=100):
    pts = [x]
    iters = 0
    while True:
        q = f(x)/df(x)
        xnext = x - q
        if xnext not in pts: pts.append(xnext); 
        iters +=1
        x = xnext
        if not((abs(f(x))>ytol) and (abs(q)>xtol) and (iters < maxiter)):
            return pts
        

In [ ]:
def f(x): return np.sin(x)
def df(x): return np.cos(x)

zs = newton_pts(f,df,1)
assert np.isclose(zs[-1], 0)
assert 5 <= len(zs) <= 6

In [ ]:
def f(x): return np.sin(x)
def df(x): return np.cos(x)

zs = newton_pts(f,df,1)
assert np.abs(zs[-2]-zs[-3]) >= 1e-8

In [ ]:
def make_f(a):
    def f(x):
        return np.exp(x)-a
    return f
def df(x): return np.exp(x)

for a in range(2,11):
    f = make_f(a)
    zs = newton_pts(f,df,0,ytol=0)
    assert np.isclose(zs[-1], np.log(a))
    assert np.abs(zs[-1]-zs[-2]) <= 1e-8
    assert np.abs(zs[-2]-zs[-3]) >= 1e-8

## Questão 2: Uma equação trigonométrica

Vamos utilizar os nossos métodos para achar uma solução para uma equação difícil de resolver "na mão":

$$ \sin(x^2) = \cos(x)^2. $$

In [ ]:
def f1(x): return np.sin(x**2)
def f2(x): return np.cos(x)**2

Faça um gráfico das funções $\sin(x^2)$ e $\cos(x)^2$ abaixo.

Com a ajuda do gráfico, defina um ponto inicial $x_0$
para o método de Newton encontrar a primeira raiz positiva da equação.

In [ ]:
t = np.linspace(0,4,1000)
plt.plot(t,f1(t), label="sin(x^2)")
plt.plot(t,f2(t), label="cos(x)^2")
plt.title("Gráfico de sin(x^2) e cos(x)^2")
plt.legend()
ax = plt.gca()

In [ ]:
x0=0.5

In [ ]:
assert len(ax.lines) == 2
assert len(ax.legend().texts) == 2
assert ax.title.get_text() != ""

Faça um gráfico mostrando a evolução do erro em $y$, em função do número de passos efetuados.

In [ ]:
eq = lambda x: f1(x) - f2(x)
deq = lambda x: np.cos(x**2)*2*x + 2*np.cos(x)*np.sin(x)
iters = np.linspace(0,5, 6)
er_y = [abs(eq(newton_pts(eq, deq, x0, ytol=0, xtol=0, maxiter=max_i)[-1])) for max_i in iters]
plt.semilogy(iters, er_y)
plt.title("Erro por número de iterações")
plt.xlabel("iteração")
plt.ylabel("erro")
ax = plt.gca()

In [ ]:
assert len(ax.lines) == 1
assert ax.title.get_text() != ""
assert ax.get_xlabel() != ""

Comente o resultado.

A precisão já começa em $10^{-1}$ com 1 iteração, com mais 2 iterações ela atinge o erro mínimo, ytol, padrão da função. Pode-se notar que o erro diminui exponencialmente até 4 iterações, onde atinge o limite.

Você acha que seria uma boa ideia fazer um gráfico do erro em $x$ nesse caso?
Porquê?

O erro em x é calculado com $|{\frac{f(x)}{df(x)}}|$, isso significa que é uma operação entre dois números resultados de outras operações feitas anteriormente, para determinar f(x) e df(x). Esses dois números já têm um erro, devido à limitação de precisão do computador, logo, fazer mais uma operação com eles, produz mais erro. Isso resulta em um erro maior do que o erro real, que seria calculado usando a raiz, se fosse conhecida. Usando o erro em y, só precisamos calcular ${f(x)}$, envolvendo menos contas feitas pelos computador, resultando em um valor mais próximo do valor real de f(x), que em módulo é o erro em y     

## Questão 3: Polinômios em `Python`

Para não precisar ficar implementando polinômio por polinômio,
implemente uma função `cria_poli(coefs)` que retorna uma (outra) função `poli(x)`,
que computa o valor do polinômio

$p(X)=$ `coefs[0]`  $+$ `coefs[1]` $X+$ `coefs[2]` $X^2+\dots+$ `coefs[-1]` $X^n$

em $X=$`x`.
As funções de polinômios do `numpy` (`poly1d`, `polyval`, `polyder`) podem lhe ser úteis :-)

No mesmo espírito, crie a função `cria_derivada_poli(coefs)`,
que retorna uma função `derivada_poli(x)` que computa a derivada do polinômio acima em $X=$`x`. 

In [ ]:
def cria_poli(l):
    def poli(x):
        p = np.poly1d([l[len(l)-1-i] for i in range(len(l))])
        return np.polyval(p,x)
    return poli

def cria_derivada_poli(l):
    p = np.poly1d([l[len(l)-1-i] for i in range(len(l))])
    def derivada_poli(x):
        der_poly = np.polyder(p)
        return np.polyval(der_poly,x)
    return derivada_poli

In [ ]:
p = cria_poli([1,2,3,4,5,6,7,8])
assert p(0) == 1
assert p(1) == 8*9/2

In [ ]:
for n in range(2,11):
    dp = cria_derivada_poli(np.linspace(0,n,n+1))
    assert dp(1) == np.sum([c**2 for c in range(n+1)])

In [ ]:
np.random.seed(21)
for n in range(2,6):
    coefs = np.random.rand(n)*100
    x = np.random.rand()*10
    p = cria_poli(coefs)
    dp = cria_derivada_poli(coefs)
    assert np.isclose(p(x+0.002), p(x)+0.002*dp(x))

## Questão 4: Achando as raízes de um polinômio

Defina o polinômio
$p(x) = 1 - \frac{1}{2}x - 4x^2 + \frac{1}{2}x^3 + \frac{3}{2}x^4 + \frac{1}{2}x^5$,
e faça um gráfico que permita visualizar satisfatoriamente a região onde se encontram as suas raízes.

Deduza um intervalo $[a,b]$ que contenha todas as raízes reais de $p(x)$.

In [ ]:
l= [1,-1/2,-4,1/2,3/2,1/2]
p = np.poly1d([l[len(l)-1-i] for i in range(len(l))])
dp = np.polyder(p)
x = np.linspace(-0.6,1.25,1000)
plt.plot(x,p(x))
plt.axhline(color="k",linewidth="0.6");

Divida o intervalo $[a,b]$ em um número bem grande de pontos.

Faça um gráfico demonstrando o número de iterações necessárias para que o método de Newton convirja,
tendo cada ponto do intervalo como valor inicial.

In [ ]:
x0 = np.linspace(-0.6,1.25,100)

def newton_pts_iters(f, df, x, xtol=1e-8, ytol=1e-8, maxiter=100):
    pts = [x]
    iters = 0
    while True:
        q = f(x)/df(x)
        xnext = x - q
        if xnext not in pts: pts.append(xnext); 
        iters +=1
        x = xnext
        if not((abs(f(x))>ytol) and (abs(q)>xtol) and (iters < maxiter)):
            return pts, iters
y = [newton_pts_iters(p, dp, x)[1] for x in x0]
plt.plot(x0, y)
plt.title("Número de iterações utilizadas por dado ponto inicial")
plt.xlabel("ponto inicial")
ax = plt.gca()

In [ ]:
assert len(ax.lines) == 1
assert ax.title.get_text() != ""
assert ax.get_xlabel() != ""

Agora, faça o gráfico das raizes para as quais o método converge,
em função do ponto inicial.

In [ ]:
pt = [newton_pts_iters(p, dp, x)[0][-1] for x in x0]
plt.plot(x0, pt)
plt.xlabel("ponto inicial")
plt.ylabel("raíz")
plt.title("Raíz encontrada por ponto inicial");


Comente os gráficos.

O gráfico do número de iterações utilizadas por ponto iniciado mostra um crescimentos entorno de dois pontos, com alguns picos ocorrendo. O segundo gráfico mostra que na maioria das vezes, para pontos próximos, a raíz encontrada é a mesma, ou a mais próxima dele, mas há pontos, fugindo deste padrão, que levam a outras raízes, nem sempre a mais próxima, nem sempre a mesma encontrada para outros pontos próximos.

## Questão 5: Outro polinômio
Repita o mesmo estudo para o polinômio

$$p(x) = 1 - \frac{1}{2}x - \frac{3}{2}x^2 + \frac{1}{2}x^3 + \frac{3}{2}x^4 + \frac{1}{2}x^5. $$

Gráfico

In [ ]:
l2= [1,-1/2,-3/2,1/2,3/2,1/2]
p2 = np.poly1d([l2[len(l)-1-i] for i in range(len(l2))])
dp2 = np.polyder(p2)
x2 = np.linspace(-2.25,1,1000)
plt.plot(x2,p2(x2))
plt.axhline(color="k",linewidth="0.6");

Iterações para encontrar as raízes, e raízes encontradas.

In [ ]:
x02 = np.linspace(-2.25,1,100)

y2 = [newton_pts_iters(p2, dp2, x)[1] for x in x02]
plt.plot(x02, y2)
plt.title("Número de iterações utilizadas por dado ponto inicial")
plt.xlabel("ponto inicial")
ax = plt.gca()

In [ ]:
pt2 = [newton_pts_iters(p2, dp2, x)[0][-1] for x in x02]
plt.plot(x02, pt2)
plt.xlabel("ponto inicial")
plt.ylabel("raíz")
plt.title("Raíz encontrada por ponto inicial");


O que mudou dessa vez? Que conclusão você tira disso?

o primeiro gráfico, completamente irregular, mostra que alguns pontos atingiram o número máximo de iterações padrão da função, e - como podemos ver no segundo gráfico - retornaram valores diferentes entre si e entre o restante dos pontos, que retornaram todos o mesmo valor para a raíz.